In [ ]:
import pandas as pd
import numpy as np
import pgeocode
import geopy
import random

In [ ]:
def gerar_lista_ajustada(n_escolas, n_candidatos):
    """Gerando uma lista aleatória para substituir a quantidade de vagas para cada escola, de acordo com o numero de candidatos."""
    # Cria uma lista inicial com valores aleatórios
    lista = [random.randint((n_candidatos // n_escolas) - 10, (n_candidatos // n_escolas)+10) for _ in range(n_escolas)]
    # Ajusta a soma da lista para que seja igual a n_candidatos
    soma_atual = sum(lista)
    diferenca = n_candidatos - soma_atual
    # Ajuste da lista para balancear a diferença
    while diferenca != 0:
        for i in range(n_escolas):
            # Ajuste gradual: adiciona ou subtrai 1 conforme a diferença restante
            if diferenca > 0 and lista[i] < 200:
                lista[i] += 1
                diferenca -= 1
            elif diferenca < 0 and lista[i] > 50:
                lista[i] -= 1
                diferenca += 1
            # Sai do loop se a diferença já for ajustada
            if diferenca == 0:
                break
    return lista

## Notebook para gerar samples de `Candidatos` e `Escolas`

### Abrindo os dados

In [2]:
# abrindo os dados dos candidatos em csv
df_candidatos = pd.read_csv('../dados/candidatos_concurso.csv', sep=',')
print(df_candidatos.shape)
df_candidatos.head()

(100000, 2)


,Nome,CEP
0,Renan Cavalcante,26525030
1,Bruna Mendonça,21230700
2,Heloísa Ramos,28922165
3,Lara Fogaça,27123290
4,Luara Aragão,27949380


In [3]:
# abrindo os dados dos locais de prova em csv
df_escolas = pd.read_csv('../dados/Escolas_Finais_1000_Unicas_CEPs_Com_Vagas.csv', sep=',')
print(df_escolas.shape)
df_escolas.head()

(358, 3)


,Nome_Escola,CEP,Vagas
0,COLÉGIO IMACULADA CONCEIÇÃO,22250040,380
1,COLEGIO LOGOSÓFICO GONZÁLEZ PECOTCHE,22290020,276
2,COLÉGIO SANTO INÁCIO,22260000,289
3,ESCOLA ADVENTISTA BOTAFOGO,22260100,278
4,ESCOLA DINAMIS,22251050,288


In [5]:
ceps_latlong = pd.read_csv("../dados/dataset_ceps_latlong.csv")
ceps_latlong.head()

,CEP,latitude,longitude
0,27525120,-22.4810134,-44.4948395
1,24903680,-22.9339527,-42.8245961
2,27525122,-22.4810134,-44.4948395
3,24903685,-22.9339527,-42.8245961
4,26083335,-22.7400001,-43.5143571


### Merge Candidatos

In [ ]:
candidatos = df_candidatos.merge(ceps_latlong, on='CEP')

# rename para Latitude e Londitude
candidatos = candidatos.rename(columns={'latitude': 'Latitude', 'longitude': 'Longitude'})

# removendo os ERROs
candidatos = candidatos[candidatos['Latitude'] != 'ERRO']
candidatos = candidatos.dropna()
# convertendo para float
candidatos['Latitude'] = candidatos['Latitude'].astype(float)
candidatos['Longitude'] = candidatos['Longitude'].astype(float)
candidatos = candidatos.groupby('CEP').first().reset_index()

# Fazendo um sample nos candidatos
n = 10000
candidatos = candidatos.sample(n).reset_index(drop=True)

# Salvando os dados
candidatos.to_csv(f"../dados/candidatos_{n}.csv", index=False)

### Merge Escolas

In [ ]:
escolas = df_escolas.merge(ceps_latlong, on='CEP')
escolas = escolas.drop_duplicates(subset='Nome_Escola')
escolas = escolas.dropna()

# filtrar todos os com nome "ERRO" na coluande latitude
escolas = escolas[escolas['latitude'] != "ERRO"]

# passando apra float
escolas['latitude'] = escolas['latitude'].astype(float)
escolas['longitude'] = escolas['longitude'].astype(float)

# sample das escolas
n = 180
escolas = escolas.sample(n).reset_index(drop=True)

n_candidatos = candidatos.shape[0]
print("Candidatos:", n_candidatos)

n_escolas = escolas.shape[0]
print("Escolas:", n_escolas)

10000
180


#### Ajustando a quantidade de vagas

In [ ]:
lista_ajustada = gerar_lista_ajustada(n_escolas, n_candidatos)
print("Lista ajustada:", lista_ajustada)
print("Soma da lista:", sum(lista_ajustada))


Lista ajustada: [67, 77, 72, 57, 61, 60, 72, 59, 63, 77, 58, 57, 58, 59, 63, 76, 64, 64, 66, 72, 67, 77, 58, 68, 62, 68, 59, 64, 73, 75, 63, 77, 74, 62, 57, 77, 74, 77, 76, 62, 77, 76, 62, 71, 56, 60, 64, 73, 76, 69, 71, 62, 76, 75, 75, 57, 57, 75, 73, 57, 76, 70, 64, 74, 59, 72, 59, 73, 66, 64, 73, 59, 73, 68, 62, 75, 76, 64, 60, 62, 57, 56, 63, 61, 66, 65, 72, 64, 58, 73, 65, 73, 75, 76, 76, 62, 62, 60, 63, 60, 65, 64, 65, 64, 69, 70, 63, 69, 57, 62, 73, 75, 69, 68, 76, 71, 64, 74, 61, 62, 69, 56, 70, 67, 65, 62, 74, 63, 64, 62, 74, 65, 71, 69, 75, 59, 63, 59, 61, 64, 71, 74, 69, 58, 68, 64, 57, 72, 64, 60]
Soma da lista: 10000


In [ ]:
# Inserindo a quantidaed de vagas nas escolas
escolas['Vagas'] = lista_ajustada
# validação
escolas['Vagas'].sum() == n_candidatos

True

In [ ]:
# rename daslatitude e longitude
escolas = escolas.rename(columns={'latitude': 'Latitude', 'longitude': 'Longitude'}) 
# salvando as escolas em csv
escolas.to_csv(f"../dados/escolas_{n}.csv", index=False)

In [ ]:
# Validando se temos escolas com o mesmo cep
escolas.groupby(['Nome_Escola'])[['CEP']].count().sort_values(by='CEP', ascending=False)

,CEP
Nome_Escola,
1/2 - COLÉGIO SAGRADO CORAÇÃO DE MARIA - T 56,1
ESCOLA MUNICIPAL MANOEL BOMFIM,1
ESCOLA MUNICIPAL ISAIAS ALVES,1
ESCOLA MUNICIPAL JAIME COSTA,1
ESCOLA MUNICIPAL JOAO KOPKE,1
...,...
ESCOLA EDEM,1
ESCOLA ELEVA,1
ESCOLA ESTADUAL ALMIRANTE PROTOGENES,1


In [ ]:
escolas.groupby(['CEP'])[['Nome_Escola']].count().sort_values(by='Nome_Escola', ascending=False)

***
***